In [1]:
import pickle
import Feature_Extraction
import pandas as pd
import numpy as np

haar family: haar
db family: db1, db2, db3, db4, db5, db6, db7, db8, db9, db10, db11, db12, db13, db14, db15, db16, db17, db18, db19, db20, db21, db22, db23, db24, db25, db26, db27, db28, db29, db30, db31, db32, db33, db34, db35, db36, db37, db38
sym family: sym2, sym3, sym4, sym5, sym6, sym7, sym8, sym9, sym10, sym11, sym12, sym13, sym14, sym15, sym16, sym17, sym18, sym19, sym20
coif family: coif1, coif2, coif3, coif4, coif5, coif6, coif7, coif8, coif9, coif10, coif11, coif12, coif13, coif14, coif15, coif16, coif17
bior family: bior1.1, bior1.3, bior1.5, bior2.2, bior2.4, bior2.6, bior2.8, bior3.1, bior3.3, bior3.5, bior3.7, bior3.9, bior4.4, bior5.5, bior6.8
rbio family: rbio1.1, rbio1.3, rbio1.5, rbio2.2, rbio2.4, rbio2.6, rbio2.8, rbio3.1, rbio3.3, rbio3.5, rbio3.7, rbio3.9, rbio4.4, rbio5.5, rbio6.8
dmey family: dmey
gaus family: gaus1, gaus2, gaus3, gaus4, gaus5, gaus6, gaus7, gaus8
mexh family: mexh
morl family: morl
cgau family: cgau1, cgau2, cgau3, cgau4, cgau5, cgau6, cgau7, 

In [2]:
def load_in_participant_data(path):
    with open(path, 'rb') as pickle_file:
        loaded_data = pickle.load(pickle_file)

    NeedToSearch = loaded_data[0]
    CorrectSearch = loaded_data[1]
    IncorrectSearch = loaded_data[2]

    return NeedToSearch, CorrectSearch, IncorrectSearch

In [3]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch


def extract_features_extended(eeg_segment):
    features = []

    # Basic statistical features for each channel
    mean_vals = np.mean(eeg_segment, axis=1)
    std_devs = np.std(eeg_segment, axis=1)
    max_vals = np.max(eeg_segment, axis=1)
    min_vals = np.min(eeg_segment, axis=1)
    skewness_vals = skew(eeg_segment, axis=1)
    kurtosis_vals = kurtosis(eeg_segment, axis=1)

    features.extend(mean_vals)
    features.extend(std_devs)
    features.extend(max_vals)
    features.extend(min_vals)
    features.extend(skewness_vals)
    features.extend(kurtosis_vals)

    # Frequency domain features using Welch's method
    for channel_data in eeg_segment:
        f, psd = welch(channel_data, fs=500, nperseg=256)
        psd_bands = [np.sum(psd[(f >= low) & (f < high)]) for (low, high) in [(0, 4), (4, 8), (8, 12), (12, 30), (30, 50)]]
        spectral_entropy = -np.sum(psd * np.log2(psd + 1e-12))  # Avoid log(0)
        peak_frequency = f[np.argmax(psd)]

        features.extend(psd_bands)
        features.append(spectral_entropy)
        features.append(peak_frequency)

    # Time-domain features
    for channel_data in eeg_segment:
        hjorth_params = [np.var(channel_data), np.var(np.diff(channel_data)), np.var(np.diff(np.diff(channel_data)))]
        zero_cross_rate = np.sum(np.diff(np.sign(channel_data)) != 0) / len(channel_data)
        waveform_length = np.sum(np.abs(np.diff(channel_data)))

        features.extend(hjorth_params)
        features.append(zero_cross_rate)
        features.append(waveform_length)

    return np.array(features)




In [4]:
def create_event_condition_features(event_condition):
    list_of_events = event_condition[0]
    list_of_events_id = event_condition[1]
    Feature_list = []

    for event in list_of_events:
        event_data = event[0]
        event_time = event[1]


        features = extract_features_extended(event_data)
        Feature_list.append(features)

    return Feature_list, list_of_events_id


In [10]:
ParticipantList = ["01", "02", "03", "04", "05","08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]

In [11]:
for participant_number in ParticipantList:

    path = fr"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\DataSegments\participant_0{participant_number}_EEG_data.pkl"

    NeedToSearch, CorrectSearch, IncorrectSearch = load_in_participant_data(path)

    NeedToSearch_features = create_event_condition_features(NeedToSearch)
    CorrectSearch_features = create_event_condition_features(CorrectSearch)
    IncorrectSearch_features = create_event_condition_features(IncorrectSearch)

    Participant_Features = {
        'NeedToSearch': NeedToSearch_features,
        'CorrectSearch': CorrectSearch_features,
        'IncorrectSearch': IncorrectSearch_features
    }

    pickle_file_path = fr'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\stat_features\participant_0{participant_number}_Features.pkl'

    # Open the file in binary write mode and use pickle.dump to save the data
    with open(pickle_file_path, 'wb') as pickle_file:
        pickle.dump(Participant_Features, pickle_file)

C:\Users\gxb18167\AppData\Local\Temp\ipykernel_12736\475317819.py:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness_vals = skew(eeg_segment, axis=1)
C:\Users\gxb18167\AppData\Local\Temp\ipykernel_12736\475317819.py:15: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_vals = kurtosis(eeg_segment, axis=1)
